In [12]:
from aidream_registration import constants
import aidream_registration.utils.cohort_utils as cu
from aidream_registration.dataloaders import AtlasImagingNiftiLoader


In [13]:
from pathlib import Path
import shutil
import numpy as np
import os

import ants

In [14]:
# the list of perfusion patients :
list_patients = cu.get_perfusion_patients()
print(fr"Number of patients: {len(list_patients)}")


Number of patients: 186


In [15]:
# the referential table :
df_ref = cu.get_referential_table(list_patients)
df_ref.head(5)


,AIDREAM_ID,cohort_ID,local_ID,perfusion,complete_validation,surgery_type,pre_RT_reference,pre_RT_TPS,valid_pre_RT_segmentation_model,valid_pre_RT_perfusion_model,Rechute_reference,Rechute_TPS,valid_Rechute_segmentation_model,valid_Rechute_perfusion_model
0,AIDREAM_1,MMI-PROB_140,201211519BP,yes,1,2.0,AC_repATLAS,RS,yes,yes,AC_repATLAS,RS,yes,yes
1,AIDREAM_10,MMI-PROB_050,201002036RR,yes,1,2.0,AC_repATLAS native,RS,yes,yes,AC_repATLAS,RS,yes,yes
2,AIDREAM_100,MMI-PROB_139,201210738EE,yes,1,0.0,AC_repATLAS,RS,yes,yes,AC_repATLAS,RS,yes,no
3,AIDREAM_102,MMI-PROB_085,201101726ZB,yes,1,2.0,AC_repATLAS,RS,yes,yes,AC_repATLAS,RS,yes,yes
4,AIDREAM_103,MMI-PROB_166,201311898RR,yes,1,2.0,AC_repATLAS,RS,yes,yes,AC_repATLAS,RS,yes,yes


In [16]:
atlas_loader_with_skull = AtlasImagingNiftiLoader(source_mri="PIPELINE")
atlas_loader_skullstripped = AtlasImagingNiftiLoader(source_mri="PIPELINE_SS")


# 1. First, register pre_RT labels : #

In [17]:
dir_workspace = constants.DIR_DEFAULT_HARD_DRIVE / "AIDREAM DATA" / "LABELS DATA" / "REGISTERED LABELS ON PRE_RT T1"
dir_workspace.mkdir(parents=True, exist_ok=True)


In [18]:
df_ref.groupby("pre_RT_reference").count()

,AIDREAM_ID,cohort_ID,local_ID,perfusion,complete_validation,surgery_type,pre_RT_TPS,valid_pre_RT_segmentation_model,valid_pre_RT_perfusion_model,Rechute_reference,Rechute_TPS,valid_Rechute_segmentation_model,valid_Rechute_perfusion_model
pre_RT_reference,,,,,,,,,,,,,
AC_repATLAS,119,119,119,119,119,119,119,119,119,117,119,119,119
AC_repATLAS native,1,1,1,1,1,1,1,1,1,1,1,1,1
GE_repNATIV,38,38,38,38,38,38,38,38,38,36,38,38,38
GE_repNativAXIALISEDnew,9,9,9,9,9,9,9,9,9,7,9,9,9
GE_repNativnew,12,12,12,12,12,12,12,12,12,12,12,12,12
"JBD : Quel repère ? Pas de crâne, jcomprends pas",1,1,1,1,1,1,1,1,1,1,1,1,1
Repère natif,1,1,1,1,1,1,1,1,1,1,1,1,1
skullstripaxialized RS recalé sur GE,5,5,5,5,5,5,5,5,5,5,5,5,5


In [20]:
# First, the patients with AC_repATLAS reference :
# We register the pre_RT T1CE on the skullstripped ATLAS pre_RT T1, using both affine and SyN registration :
# Then, we apply the same transformation to the pre_RT labels :

list_patients_1 = df_ref.loc[df_ref["pre_RT_reference"] == "AC_repATLAS", "AIDREAM_ID"].tolist()
print(fr"Nbr of patients with AC_repATLAS reference: {len(list_patients_1)}")

for i, patient in enumerate(list_patients_1):

    print(fr"Processing {i + 1}/{len(list_patients_1)}: {patient} .........")

    # Load the ATLAS pre_RT T1 image :
    ants_atlas_t1 = atlas_loader_skullstripped.load_mri(patient, "pre_RT", "T1")

    # Load the reference of the pre_RT labels :
    path_labels_ref_t1ce = (constants.DIR_DEFAULT_HARD_DRIVE
                            / "AIDREAM DATA"
                            / "LABELS DATA"
                            / "NIFTI"
                            / patient
                            / "pre_RT"
                            / "SOURCE IMAGING"
                            / f"{patient}_pre_RT_T1CE.nii.gz")
    assert path_labels_ref_t1ce.exists(), f"File {path_labels_ref_t1ce} does not exist"

    ants_labels_ref_t1ce = ants.image_read(str(path_labels_ref_t1ce))

    # Register the pre_RT T1CE on the ATLAS pre_RT T1 image :
    for type_transform in ["Affine", "SyN"]:
        print(f"Processing {type_transform} .........")

        path_transform = dir_workspace / "pre_RT" / patient / "transforms" / f"{patient}_{type_transform}_"
        path_transform.parent.mkdir(exist_ok=True, parents=True)
        
        print(fr"Registration of {patient} with {type_transform} .........")
        
        my_tx = ants.registration(fixed=ants_atlas_t1,
                                  moving=ants_labels_ref_t1ce,
                                  type_of_transform=type_transform,
                                  outprefix=str(path_transform))
        
        print(fr"Applying the transformation to the pre_RT T1CE of {patient}  with {type_transform}.........")
        
        warped_label_ref_t1ce = ants.apply_transforms(fixed=ants_atlas_t1,
                                                      moving=ants_labels_ref_t1ce,
                                                      transformlist=my_tx['fwdtransforms'],
                                                      interpolator="bSpline")
        
        path_warped_label_ref_t1ce = dir_workspace / "pre_RT" / patient / "REGISTERED SOURCE IMAGING" / f"{patient}_pre_RT_T1CE_{type_transform}.nii.gz"
        path_warped_label_ref_t1ce.parent.mkdir(exist_ok=True, parents=True)
        
        ants.image_write(warped_label_ref_t1ce, str(path_warped_label_ref_t1ce))
        
        for label in ["L1", "L2", "L3", "L4", "L5"]:
            
            print(fr"Applying the transformation to the label {label} of {patient}  with {type_transform}.........")
            
            path_warped_label = dir_workspace / "pre_RT" / patient / type_transform / f"{patient}_pre_RT_{label}_{type_transform}.nii.gz"
            path_warped_label.parent.mkdir(exist_ok=True, parents=True)
            
            path_label = (constants.DIR_DEFAULT_HARD_DRIVE
                          / "AIDREAM DATA"
                          / "LABELS DATA"
                          / "REFACTORED LABELS NIFTI"
                          / patient
                          / "pre_RT"
                          / f"{patient}_pre_RT_{label}.nii.gz")
            if not path_label.exists():
                
                ants_label = ants.from_numpy(np.zeros(ants_atlas_t1.shape), origin=ants_atlas_t1.origin,
                                             spacing=ants_atlas_t1.spacing, direction=ants_atlas_t1.direction)
                
                ants.image_write(ants_label, str(path_warped_label))
            
            else:
                
                ants_label = ants.image_read(str(path_label))
                
                warped_label = ants.apply_transforms(fixed=ants_atlas_t1,
                                                     moving=ants_label,
                                                     transformlist=my_tx['fwdtransforms'],
                                                     interpolator="genericLabel")
                
                ants.image_write(warped_label, str(path_warped_label))
            

        



Nbr of patients with AC_repATLAS reference: 119
Processing 1/119: AIDREAM_1 .........
Processing Affine .........
Registration of AIDREAM_1 with Affine .........
Applying the transformation to the pre_RT T1CE of AIDREAM_1  with Affine.........
Applying the transformation to the label L1 of AIDREAM_1  with Affine.........
Applying the transformation to the label L2 of AIDREAM_1  with Affine.........
Applying the transformation to the label L3 of AIDREAM_1  with Affine.........
Applying the transformation to the label L4 of AIDREAM_1  with Affine.........
Applying the transformation to the label L5 of AIDREAM_1  with Affine.........
Processing SyN .........
Registration of AIDREAM_1 with SyN .........
Applying the transformation to the pre_RT T1CE of AIDREAM_1  with SyN.........
Applying the transformation to the label L1 of AIDREAM_1  with SyN.........
Applying the transformation to the label L2 of AIDREAM_1  with SyN.........
Applying the transformation to the label L3 of AIDREAM_1  wi